In [16]:
import pandas as pd
import json
import random


In [17]:
df = pd.read_csv('veiculos_2022.csv')

features = []

for index, row in df.iterrows():

    if row['LATITUDE'] < 0:
        
        feature = {
            'type': 'Feature',
            'geometry': {
                'type': 'Point',
                'coordinates': [row['LONGITUDE'], row['LATITUDE']]
            },
            'properties': {
                'clearance': index
            }, 
            "assaltos": random.randint(1,10)
        }
        features.append(feature)

    else:
        continue
        


geojson = {
    'type': 'FeatureCollection',
    'features': features
}



In [18]:
geojson

{'type': 'FeatureCollection',
 'features': [{'type': 'Feature',
   'geometry': {'type': 'Point',
    'coordinates': [-46.4195544052641, -23.562599528566]},
   'properties': {'clearance': 0},
   'assaltos': 10},
  {'type': 'Feature',
   'geometry': {'type': 'Point',
    'coordinates': [-46.3995427585613, -23.5591913820374]},
   'properties': {'clearance': 1},
   'assaltos': 8},
  {'type': 'Feature',
   'geometry': {'type': 'Point',
    'coordinates': [-46.528715561, -23.584177993]},
   'properties': {'clearance': 2},
   'assaltos': 2},
  {'type': 'Feature',
   'geometry': {'type': 'Point',
    'coordinates': [-46.7689812638353, -23.5889454432406]},
   'properties': {'clearance': 3},
   'assaltos': 4},
  {'type': 'Feature',
   'geometry': {'type': 'Point',
    'coordinates': [-46.647850588, -23.7064335549999]},
   'properties': {'clearance': 4},
   'assaltos': 9},
  {'type': 'Feature',
   'geometry': {'type': 'Point',
    'coordinates': [-46.5975415658285, -23.6029833840723]},
   'proper

In [19]:
with open('clearance_map.json', 'w') as f:
    json.dump(geojson, f)

### criar um leve deslocamento nas coordenadas iguais para evitar sobreposição
Isso corrige um bug da biblioteca turf.js e mapbox de não contar os obstaculos sobrepostos

In [21]:
from geopy.distance import geodesic
import json

In [29]:
# Abrir o arquivo JSON
with open('clearance_map.json') as f:
    data = json.load(f)

# Criar um dicionário para armazenar as coordenadas que já foram deslocadas
deslocadas = {}
# Criar um conjunto para armazenar todas as coordenadas (antes e depois do deslocamento)
todas_coords = set()

# Iterar sobre os recursos no arquivo JSON
for feature in data['features']:
    # Obter as coordenadas do recurso
    coords = feature['geometry']['coordinates']
    # Tentar deslocar as coordenadas
    while tuple(coords) in todas_coords:
        # Se as coordenadas já foram usadas, calcular um novo deslocamento
        # Se as coordenadas já foram usadas, calcular um novo deslocamento
        desloc = geodesic(meters=1).destination(coords, 45).longitude - coords[0], geodesic(meters=1).destination(coords, 45).latitude - coords[1]
        coords = (coords[0] + desloc[0], coords[1] + desloc[1])
    # Registrar as coordenadas originais no conjunto de todas as coordenadas
    todas_coords.add(tuple(feature['geometry']['coordinates']))
    # Registrar as novas coordenadas no conjunto de todas as coordenadas
    todas_coords.add(tuple(coords))
    # Registrar as novas coordenadas no dicionário de coordenadas deslocadas
    deslocadas[tuple(feature['geometry']['coordinates'])] = coords
    # Atualizar as coordenadas no recurso
    feature['geometry']['coordinates'] = list(coords)

# Salvar as alterações no arquivo JSON
with open('seu_arquivo_modificado.json', 'w') as f:
    json.dump(data, f)